In [ ]:
import boto3


In [47]:
# Submit a batch job
def submit_job(job_name, job_queue, job_definition, job_command, parameters):
    client = boto3.client('batch')
    response = client.submit_job(
        jobName=job_name,
        jobQueue=job_queue,
        jobDefinition=job_definition,
        containerOverrides={
            'command': job_command
        },
        parameters=parameters,
        schedulingPriorityOverride=99,
        shareIdentifier="alex",
    )
    return response['jobId']

# Submit an array job
def submit_array_job(job_name, job_queue, job_definition, job_command, array_properties):
    client = boto3.client('batch')
    response = client.submit_job(
        jobName=job_name,
        jobQueue=job_queue,
        jobDefinition=job_definition,
        containerOverrides={
            'command': job_command
        },
        arrayProperties=array_properties,
        schedulingPriorityOverride=99,
        shareIdentifier="alex",
    )
    return response['jobId']

# Get the status of a job
def get_job_status(job_id):
    client = boto3.client('batch')
    response = client.describe_jobs(
        jobs=[job_id]
    )
    return response['jobs'][0]['status']

def get_cloudwatch_logs(job_id):
    client = boto3.client('batch')
    response = client.describe_jobs(
        jobs=[job_id]
    )
    log_stream_name = response['jobs'][0]['container']['logStreamName']

    logs_client = boto3.client('logs')
    log_group_name = f'/aws/batch/auspatious-ldn'

    response = logs_client.get_log_events(
        logGroupName=log_group_name,
        logStreamName=log_stream_name,
        startFromHead=True
    )

    events = response['events']
    for event in events:
        print(event['message'])

In [ ]:
job_name = 'test-job'
job_queue = 'LowPriorityFargate'
job_definition = 'auspatious-ldn'
# job_command = ['aws', 's3', 'ls', 's3://data.ldn.auspatious.com/']
job_command = [
    "ldn-processor",
    "--tile", "Ref::tile",
    "--year", "Ref::year",
    "--version", "Ref::version",
    "Ref::extra_options"
]
parameters = {
    "tile": "238,47",
    "year": "2023",
    "version": "0.1.0",
    "extra_options": "--overwrite ",

}

job_id = submit_job(job_name, job_queue, job_definition, job_command, parameters)
print('Job submitted with id:', job_id)

status = get_job_status(job_id)
print('Job status:', status)

In [ ]:
# Get status of job every 10 seconds
while status != 'SUCCEEDED':
    new_status = get_job_status(job_id)
    if new_status != status:
        print('Job status:', status)
        status = new_status

    import time
    time.sleep(10)
else:
    print('Job completed successfully')

In [ ]:
job_name = 'test-job'
job_queue = 'LowPriorityFargate'
job_definition = 'auspatious-ldn'
# job_command = ['aws', 's3', 'ls', 's3://data.ldn.auspatious.com/']
job_command = ["echo", 'Ref::test']
array_properties = {
    'size': 2
}
job_id = submit_array_job(job_name, job_queue, job_definition, job_command, array_properties=array_properties)
print('Job submitted with id:', job_id)

status = get_job_status(job_id)
print('Job status:', status)

# Get status of job every 10 seconds
while status != 'SUCCEEDED':
    status = get_job_status(job_id)
    import time
    time.sleep(10)
else:
    print('Job completed successfully')

In [ ]:
get_cloudwatch_logs(job_id)